In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:

import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from src.data_processor.phase_2.prob1.v13 import  Phase2Prob1FeatureProcessor
import numpy as np
from collections import Counter

In [3]:
df = pd.read_parquet("D:/Data/MLOPS_2023/data_phase-2/phase-2/prob-1/raw_train.parquet")

In [4]:
df.to_csv('output.csv',index=False)

In [5]:
df.groupby('feature3')['feature4'].value_counts()

feature3  feature4
-         FIN         16786
          INT         14111
          CON          2273
          REQ           872
          RST            17
          ECO             3
          ACC             1
dhcp      INT            25
          CON            13
dns       INT         13722
          CON          2532
          REQ            50
          FIN            17
ftp       FIN          1122
          CON             6
ftp-data  FIN          1259
          CON             1
http      FIN          6533
          CON             6
          RST             1
irc       FIN             8
pop3      FIN           377
          CON             3
radius    CON             3
          INT             3
smtp      FIN          1696
          CON            11
snmp      INT            21
ssh       FIN           344
          RST             2
ssl       FIN            23
Name: feature4, dtype: int64

In [6]:
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

(43405, 42)

In [7]:
df.reset_index(drop=True, inplace=True)
x = df.groupby(['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
                'feature7', 'feature8', 'feature9', 'feature10', 'feature11',
                'feature12', 'feature13', 'feature14', 'feature15', 'feature16',
                'feature17', 'feature18', 'feature19', 'feature20', 'feature21',
                'feature22', 'feature23', 'feature24', 'feature25', 'feature26',
                'feature27', 'feature28', 'feature29', 'feature30', 'feature31',
                'feature32', 'feature33', 'feature34', 'feature35', 'feature36',
                'feature37', 'feature38', 'feature39', 'feature40', 'feature41']).apply(lambda x : list(x.index)).reset_index()
a = x[x[0].apply(lambda x: len(x)>1)]
blacklist = []
for t in a[0].values:
    blacklist.extend(t)
    index = np.array(blacklist)
not_index = np.setdiff1d(df.index.to_numpy(), index)
df = df.iloc[not_index]
df.reset_index(inplace=True, drop=True)
df['label'].value_counts()

1    21787
0    21480
Name: label, dtype: int64

In [20]:
HyperParameters={'n_estimators':100,
                 'learning_rate':0.5,
                 'max_depth':4,
                 'colsample_bytree':0.7,
                 'subsample':0.8,
                 'reg_alpha':1,
                 'reg_lambda':0,
                 'random_state':42}

hyper_parameters = HyperParameters.copy()

In [21]:
processor = Phase2Prob1FeatureProcessor()
new_df = processor.fit_transform(df)
kfold = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
FEATURES = processor.data_features['features']
categorical = processor.data_features['categorical_features']
TARGET = 'label'
print(new_df.shape, df.shape)

models = []
scores = []
oofs = np.zeros(df.shape[0])
for i, (train_idx, valid_idx) in enumerate(kfold.split(new_df, df['label'])):
    X_train = new_df.iloc[train_idx][FEATURES]
    X_valid = new_df.iloc[valid_idx][FEATURES]
    y_train = df.iloc[train_idx][TARGET]
    y_valid = df.iloc[valid_idx][TARGET]
    # model = LogisticRegression(max_iter=7000)
    # model.fit(X_train, y_train)
    model = LGBMClassifier(**hyper_parameters)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_valid, y_valid)],
              eval_metric=["logloss", "auc"],
              categorical_feature=categorical,
              early_stopping_rounds=50,
              verbose=50)
    # model = CatBoostClassifier(iterations=100, learning_rate=0.05, max_depth=7, subsample=0.8, random_state=42,eval_metric='AUC',class_weights={1:1.5,0:1})
    # model.fit(X_train, y_train,
    #           cat_features=categorical,
    #           eval_set=[(X_train, y_train), (X_valid, y_valid)])
    models.append(model)
    y_pred_proba = model.predict_proba(X_valid)[:,1]
    y_pred = (y_pred_proba>0.5).astype(int)
    oofs[valid_idx] = y_pred_proba

    print(f"fold {i} : {roc_auc_score(y_valid, y_pred_proba)}")
    print(classification_report(y_valid, y_pred))
    scores.append(roc_auc_score(y_valid, y_pred_proba))

print(np.mean(scores), np.std(scores))

(43267, 10) (43267, 42)
[50]	training's binary_logloss: 0.159929	training's auc: 0.9839	valid_1's binary_logloss: 0.180547	valid_1's auc: 0.978375
[100]	training's binary_logloss: 0.14162	training's auc: 0.987881	valid_1's binary_logloss: 0.177287	valid_1's auc: 0.979255
fold 0 : 0.9792549586667384
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      4296
           1       0.92      0.90      0.91      4358

    accuracy                           0.91      8654
   macro avg       0.91      0.91      0.91      8654
weighted avg       0.91      0.91      0.91      8654

[50]	training's binary_logloss: 0.158947	training's auc: 0.984248	valid_1's binary_logloss: 0.177653	valid_1's auc: 0.979635
[100]	training's binary_logloss: 0.140055	training's auc: 0.988199	valid_1's binary_logloss: 0.175479	valid_1's auc: 0.979988
fold 1 : 0.9799877876086531
              precision    recall  f1-score   support

           0       0.91      0.91      

In [22]:
print(roc_auc_score(df['label'],oofs))
oofs[df[df.feature3=='ssl'].index] = 1
print(roc_auc_score(df['label'],oofs))

0.9799519614698565
0.9799533632249049


In [23]:
gmodel = LGBMClassifier(**models[0].get_params())
gmodel.fit(new_df[FEATURES], df[TARGET],
           eval_set=[(new_df[FEATURES],df[TARGET])],
           eval_metric=["logloss", "auc"],
           categorical_feature=categorical,
           verbose=50)

with open(f'../checkpoints/phase-2/prob-1/v1.pkl','wb') as file:
    pickle.dump(gmodel, file)

[50]	valid_0's binary_logloss: 0.171369	valid_0's auc: 0.982318
[100]	valid_0's binary_logloss: 0.149677	valid_0's auc: 0.986651


In [24]:
var_count_train = Counter(df['feature3'])
with open(f'../checkpoints/phase-2/prob-1/var_count.pkl', 'wb') as file:
    pickle.dump(var_count_train, file)

In [25]:
feats = pd.DataFrame({'feature':gmodel.feature_name_, 'score':gmodel.feature_importances_}).sort_values('score',ascending=False)

In [26]:
feats.feature[:10].values

array(['feature7', 'feature26', 'feature23', 'feature11', 'feature40',
       'feature35', 'feature15', 'feature27', 'feature9', 'feature29'],
      dtype=object)